# Download audio from Youtube

In [ ]:
OPENAI_API_KEY = ""

In [ ]:
%pip install pytube
%pip install pydub
%pip install openai

In [ ]:
from pytube import YouTube
import os
import math 
from pydub import AudioSegment
from openai import OpenAI

In [ ]:
def download_audio_from_youtube(yt_url):
    yt = YouTube(yt_url)
    return yt.streams.filter(only_audio=True).first().download()


In [ ]:
mp4_file = download_audio_from_youtube("https://www.youtube.com/watch?v=VHYg1P8lA9o")

# Splitting audio

In [ ]:
def split_audio_data(mp4_file, min_per_split):
    song = AudioSegment.from_file(mp4_file, "mp4")

    path = os.path.splitext(mp4_file)[0]
    
    try: 
        if not os.path.exists(path):
            os.mkdir(path)
    except OSError as error: 
        print(error)
    
    total_mins = math.ceil(song.duration_seconds / 60)
    
    for i in range(0, total_mins, min_per_split):
        file_to_export = os.path.join(path, f"{os.path.splitext(mp4_file)[0]}_{int(i / min_per_split + 1)}.mp4")
        from_min = i * 60 * 1000
        to_min = (i + min_per_split) * 60 * 1000
        split_audio = song[from_min:to_min]
        split_audio.export(file_to_export, format="mp4")
        print(f"Created: {file_to_export}")


##### Spliting all audio from mp4 video each 10 minutes to send it to Open AI API for transcribing it

In [ ]:
split_audio_data(os.path.basename(mp4_file), min_per_split = 10)

# Transcribe audio

In [ ]:
def transcribe_audios_from_folder_path(folder_path):  
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    client = OpenAI(
        api_key=OPENAI_API_KEY
    )

    transcription_path = os.path.join(folder_path, "transcriptions")
    try: 
        if not os.path.exists(transcription_path):
            os.mkdir(transcription_path)
    except OSError as error: 
        print(error)

    for file in files:
        try:
            with open(os.path.join(folder_path, file), "rb") as audio_file:
                transcript = client.audio.transcriptions.create(
                    model = "whisper-1", 
                    file = audio_file 
                )
            
            with open(os.path.join(folder_path, "transcriptions", f"{os.path.splitext(file)[0]}.txt"), 'w') as f:
                f.write(transcript.text)
        except Exception as e:
            print(e)
            
def join_transcriptions(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    file_name = os.path.splitext(files[0])[0].split("_")[0]
    
    transcriptions = []
    for file in files:
        with open(os.path.join(folder_path, file), "r") as f:
            contents = f.read()
            transcriptions.append(contents)
    
    with open(os.path.join(folder_path, f"{file_name}_full.txt"), 'w') as f:
        f.write(" ".join(transcriptions))

In [ ]:
mp4_file = "Antonio Escohotado en diálogo con Ernesto Castro.mp4"

path = f"{os.path.splitext(mp4_file)[0]}\\"

transcribe_audios_from_folder_path(path)


# Join all transcriptions

In [ ]:
mp4_file = "Antonio Escohotado en diálogo con Ernesto Castro.mp4"

path = f"{os.path.splitext(mp4_file)[0]}\\"
path = os.path.join(path, "transcriptions")

join_transcriptions(path)

# Chat GPT processing

In [ ]:
mp4_file = "Antonio Escohotado en diálogo con Ernesto Castro.mp4"

suffix_file_name = os.path.splitext(mp4_file)[0]
path = f"{suffix_file_name}\\"
text_file = os.path.join(path, "transcriptions", "Antonio Escohotado en diálogo con Ernesto Castro_full.txt")

f = open(text_file, "r")
text_contents = f.read()
f.close()

client = OpenAI(
    api_key=OPENAI_API_KEY
)

response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": """
            You are a helpful assistant.
            Given an interview provided by the user delimited by triple backticks, 
            your task is to create a detailed and concise summary of an interview, 
            highlighting the key points, opinions and perspectives of the interviewee, 
            important quotes, relevant context and background information, conclusions of the interviewer, 
            and any significant additional comments or reactions.
        """},
        {"role": "user", "content": f"```{text_contents}```"}
    ]
)


In [ ]:

response_message = response.choices[0].message.content
print(response_message)

In [ ]:
with open(os.path.join(path, "transcriptions", "Antonio Escohotado en diálogo con Ernesto Castro_result.txt"), 'w') as f:
    f.write(response_message)